In [4]:
import pandas as pd
import json
import string 

In [5]:
df = pd.read_excel(r'C:\Users\Mrulay\OneDrive - University of Windsor\uWindsor\COMP 8730 - NLP\Assignments\3. Lexical vs Vector Semantics\data\SimLex-999.xlsx')
df

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,vanish,disappear,V,9.80,2.77,2.86,2,2.76,1,0.46
1,quick,rapid,A,9.70,2.89,3.07,2,1.38,1,1.14
2,creator,maker,N,9.62,3.07,3.90,2,1.51,1,1.40
3,stupid,dumb,A,9.58,1.75,2.36,1,5.26,1,1.48
4,insane,crazy,A,9.57,1.77,2.37,1,2.09,1,0.92
...,...,...,...,...,...,...,...,...,...,...
994,ankle,window,N,0.30,4.81,4.86,4,0.00,0,1.80
995,princess,island,N,0.30,4.72,4.96,4,0.00,0,1.36
996,container,mouse,N,0.30,4.85,4.83,4,0.00,0,0.95
997,new,ancient,A,0.23,2.81,2.04,1,0.20,0,0.46


In [6]:
df = df.drop(df.columns[2], axis=1) 
df = df.drop(df.columns[3:], axis=1) 

In [7]:
SimLex = []
for i, row in df.iterrows():
    SimLex.append(row['word1'])
    SimLex.append(row['word2'])
SimLexWords = list(set(SimLex))

In [8]:
import pickle

with open('simSet.pkl', 'wb') as f:
    pickle.dump(SimLexWords, f)

In [9]:
def simUtil(ind):
    retList = []
    for i in ind:
        if i%2 == 0:
            retList.append(SimLex[i+1])
        else:
            retList.append(SimLex[i-1])
    return retList

def find_similarities(word):
    retDict = {}
    mainWords = simUtil([i for i, x in enumerate(SimLex) if x == word])
    lval = 10
    levels = [lval]*len(mainWords)
    for i, w in enumerate(mainWords):
        lval -= 1
        if i > 10:
            break
        if len(set(mainWords))<=10:
            newlis = simUtil([i for i, x in enumerate(SimLex) if x == w])
            mainWords.extend(newlis)
            applvl = [lval]*len(newlis)
            levels.extend(applvl)
        else:
            break
    visited = []
    for i, w in enumerate(mainWords):
        if w not in visited:
            visited.append(i)
    if len(visited)>10:
        visited = visited[0:11]
    for i in visited: 
        retDict[mainWords[i]] = levels[i] 
    return retDict
    

In [10]:
simDict = {}
for w in SimLexWords:
    x = find_similarities(w)
    if w in x.keys():
        del x[w]
    simDict[w] = x
simDict


{'inexpensive': {'cheap': 7, 'dirty': 5, 'narrow': 7, 'long': 4},
 'realize': {'discover': 6,
  'explore': 5,
  'originate': 9,
  'ignore': 8,
  'begin': 6},
 'squad': {'army': 6, 'navy': 4, 'legion': 5},
 'word': {'phrase': 10,
  'literature': 10,
  'dictionary': 10,
  'page': 10,
  'clue': 10,
  'newspaper': 10,
  'book': 8,
  'definition': 7},
 'bus': {'vehicle': 7, 'cab': 5, 'taxi': 8, 'car': 8, 'carriage': 5},
 'person': {'animal': 3, 'beast': 5},
 'drawer': {'clothes': 7,
  'fabric': 9,
  'cloth': 9,
  'coat': 9,
  'jacket': 9,
  'closet': 9,
  'button': 9,
  'linen': 8},
 'student': {'pupil': 6,
  'professor': 7,
  'president': 9,
  'doctor': 8,
  'politician': 6,
  'mayor': 6},
 'satisfy': {'please': 7,
  'comprehend': 7,
  'entertain': 9,
  'agree': 9,
  'plead': 9,
  'know': 8,
  'read': 8},
 'replace': {'restore': 7,
  'bring': 6,
  'carry': 7,
  'send': 7,
  'come': 7,
  'complain': 7},
 'fraud': {'crime': 6, 'theft': 4, 'violation': 5},
 'beef': {'cattle': 6, 'cow': 5, 'sh

In [11]:
with open('simLex_similarities.json', 'w') as fp:
    json.dump(simDict, fp, indent=4)